In [242]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [243]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import nltk # just for tokenization
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import random
import gradio 
import string

random.seed(42)

In [244]:
def load_embedding_model():
    """ Load GloVe Vectors
        Return:
            wv_from_bin: All 400000 embeddings, each lengh 200
    """
    import gensim.downloader as api
    wv_from_bin = api.load("word2vec-google-news-300")
    print("Loaded vocab size %i" % len(list(wv_from_bin.index_to_key)))
    return wv_from_bin
wv_from_bin = load_embedding_model()
# wv_from_bin = load_embedding_model()

Loaded vocab size 3000000


In [246]:
raw_df = pd.read_csv("../../data/MultiLabel.csv")
print(raw_df.head())

           artist    genre                  title                    album  \
0         Nirvana     Rock  You Know You’re Right                  Nirvana   
1   Damian Marley   Reggae             Here We Go               Stony Hill   
2  The Mission UK     Rock                   Jade  Another Fall from Grace   
3            UB40   Reggae       Food For Thought              Signing Off   
4     Johnny Cash  Country   I’ve Been Everywhere   American II: Unchained   

     year                                             lyrics  \
0  2002.0  I will never bother you\r\nI will never promis...   
1  2017.0  Here we go\r\nMy big ego is gonna get me in tr...   
2  2016.0  She came as Lolita dressed as Venus\r\nAnd ado...   
3  1980.0  Ivory Madonna, dying in the dust\r\nWaiting fo...   
4  1996.0  I was totin' my pack along the dusty Winnemucc...   

                                       labels  
0                           Calmness, Sadness  
1                              Power, Tension  
2 

In [247]:
len(raw_df)

1160

In [248]:
unique_emotion_labels = set()

In [250]:
[[unique_emotion_labels.add(label.strip()) for label in labels.split(",") ]for labels in raw_df["labels"]]
# raw_df["labels"].unique()

[[None, None],
 [None, None],
 [None, None, None, None],
 [None, None, None],
 [None, None, None],
 [None, None, None],
 [None, None, None, None],
 [None, None, None, None],
 [None, None, None],
 [None, None, None],
 [None, None, None, None, None],
 [None, None, None],
 [None, None, None],
 [None, None, None, None],
 [None, None, None, None],
 [None, None, None],
 [None, None],
 [None, None, None, None],
 [None, None, None],
 [None, None, None],
 [None, None, None, None],
 [None, None, None],
 [None, None],
 [None, None, None, None],
 [None, None],
 [None, None, None, None],
 [None, None, None, None],
 [None, None],
 [None, None, None],
 [None, None, None],
 [None, None],
 [None, None, None, None],
 [None, None, None],
 [None, None, None, None],
 [None, None, None],
 [None, None],
 [None, None, None, None],
 [None, None],
 [None, None, None, None, None],
 [None, None, None],
 [None, None],
 [None, None, None],
 [None, None, None],
 [None, None],
 [None, None],
 [None, None, None, None,

In [251]:
unique_emotion_labels

{'Amazement',
 'Calmness',
 'Joyful activation',
 'Nostalgia',
 'Power',
 'Sadness',
 'Solemnity',
 'Tenderness',
 'Tension'}

In [252]:
def tokenize(lyric: str) -> list[str]:
    # lowercase the text, remove stop words, punctuation and keep only the words
    tokens = nltk.tokenize.word_tokenize(lyric.lower())
    stop_words = stopwords.words("english") + list(string.punctuation)
    lemmatizer = WordNetLemmatizer()
    alpha_tokens = [lemmatizer.lemmatize(token) for token in tokens if token.isalpha() and token not in stop_words]

    return alpha_tokens

In [253]:
# go through each lyrics, tokenize it, vectorize each word, then combine all of them into single average vector and store it in the list
lyrics = raw_df["lyrics"]
lyrics_embeddings = []
unsupported_tokens = set()
label_embedding_map = {} # dict{str: np.array([])}
for lyric in tqdm(lyrics):
    lyric_vector = np.zeros(300)
    for token in tokenize(lyric):
        try:
            lyric_vector += wv_from_bin.get_vector(token.lower())
        except KeyError as e:
            # if the word is not present in the glove then key error is raised, so handle the exception and move on
            unsupported_tokens.add(token)
            continue
    lyrics_embeddings.append(lyric_vector)


lyrics_embeddings = np.stack(lyrics_embeddings)
scaled_lyrics_embeddings = lyrics_embeddings / np.linalg.norm(lyrics_embeddings, axis=1, keepdims=True)

100%|██████████| 1160/1160 [00:04<00:00, 277.19it/s]


# Prediction

In [254]:
def vectorise(lyrics: str) -> np.array:
    tokens = tokenize(lyrics)
    lyric_vector = np.zeros(300)
    for token in tokens:
        try:
            lyric_vector += wv_from_bin.get_vector(token.lower())
        except:
            continue
    return lyric_vector / np.linalg.norm(lyric_vector)
    

In [255]:
test_lyric = """
I walk a lonely road
The only one that I have ever known
Don't know where it goes
But it's only me, and I walk alone
I walk this empty street
On the boulevard of broken dreams
Where the city sleeps
And I'm the only one, and I walk alone
I walk alone, I walk alone
I walk alone, and I walk a
My shadow's the only one that walks beside me
My shallow heart's the only thing that's beating
Sometimes I wish someone out there will find me
'Til then I walk alone
Ah ah ah ah ah
Ah ah ah
I'm walking down the line
That divides me somewhere in my mind
On the border line of the edge
And where I walk alone
Read between the lines
What's fucked up and every thing's all right
Check my vital signs to know I'm still alive
And I walk alone
I walk alone, I walk alone
I walk alone and I walk a
My shadow's the only one that walks beside me
My shallow heart's the only thing that's beating
Sometimes I wish someone out there will find me
'Til then I walk alone
Ah ah ah ah ah
Ah ah
I walk alone, and I walk a
I walk this empty street
On the boulevard of broken dreams
Where the city sleeps
And I'm the only one, and I walk a
My shadow's the only one that walks beside me
My shallow heart's the only thing that's beating
Sometimes I wish someone out there will find me
'Til then I walk alone
"""

In [256]:
test1_vector = vectorise(test_lyric)

In [257]:
scaled_lyrics_embeddings.shape

(1160, 300)

In [258]:
test1_vector.shape

(300,)

In [259]:
def distance(metric: str, embedding_matrix: np.ndarray, test_vector: np.ndarray) -> np.ndarray:
    if metric == "cosine":
            
        dot_product = np.dot(embedding_matrix, test_vector)

        # Compute magnitudes
        embedding_magnitudes = np.linalg.norm(embedding_matrix, axis=1)
        test_vector_magnitude = np.linalg.norm(test_vector)

        # Compute cosine similarity
        cosine_similarity = dot_product / (embedding_magnitudes * test_vector_magnitude)
        return cosine_similarity
    elif metric == "euclidean":
        distances = np.linalg.norm(scaled_lyrics_embeddings - test_vector, axis=1)
        return distances
    else:
        raise Exception(f"Invalid parameter value {metric}")

In [260]:
unique_emotion_labels

{'Amazement',
 'Calmness',
 'Joyful activation',
 'Nostalgia',
 'Power',
 'Sadness',
 'Solemnity',
 'Tenderness',
 'Tension'}

In [261]:
test_vector = np.array([0.5, 0.6, 0.7])
test_vector.shape

(3,)

In [262]:
similarity = distance(metric="euclidean", embedding_matrix=scaled_lyrics_embeddings, test_vector= test1_vector.reshape(1,-1))

In [263]:
cs = distance(metric="cosine", embedding_matrix=scaled_lyrics_embeddings, test_vector= test1_vector)

In [265]:
def cosine_similarity(X, centroids):
    # Compute cosine similarity between each data point and each centroid
    dot_product = np.dot(X, centroids.T)
    norms_X = np.linalg.norm(X, axis=1)[:, np.newaxis]
    norms_centroids = np.linalg.norm(centroids, axis=1)
    cosine_similarities = dot_product / (norms_X * norms_centroids)
    return cosine_similarities

In [266]:
real = cosine_similarity(scaled_lyrics_embeddings, test1_vector.reshape(1,-1))[:,0]
raw_df.loc[(-real).argsort()[:3]]["labels"]

1042                          Sadness, Solemnity, Tension
721     Calmness, Nostalgia, Sadness, Solemnity, Tende...
773        Amazement, Calmness, Power, Solemnity, Tension
Name: labels, dtype: object

In [267]:
real.max(), real.argmax()

(0.8401738247892171, 1042)

In [268]:
cs.max(), cs.argmax()

(0.8401738247892171, 1042)

In [269]:
raw_df.loc[(-cs).argsort()[:3]]["labels"]

1042                          Sadness, Solemnity, Tension
721     Calmness, Nostalgia, Sadness, Solemnity, Tende...
773        Amazement, Calmness, Power, Solemnity, Tension
Name: labels, dtype: object

In [270]:
similarity.max(), similarity.argmax()

(1.1170934657770546, 112)

In [271]:
(-similarity).argsort()

array([ 112,  159,  407, ...,  773,  721, 1042], dtype=int64)

In [272]:
raw_df.loc[(-similarity).argsort()[:10]]["labels"]

112    Calmness, Joyful activation, Power, Sadness, T...
159                                     Sadness, Tension
407                                  Amazement, Calmness
628      Amazement, Nostalgia, Power, Solemnity, Tension
970               Nostalgia, Sadness, Solemnity, Tension
552                         Amazement, Joyful activation
404                   Calmness, Joyful activation, Power
509         Joyful activation, Power, Solemnity, Tension
284    Calmness, Joyful activation, Nostalgia, Sadnes...
78        Amazement, Joyful activation, Power, Solemnity
Name: labels, dtype: object

# Combining embeddings for each emotion

In [273]:
# go through each lyrics, tokenize it, vectorize each word, then combine all of them into single average vector and store it in the list
lyrics = raw_df["lyrics"]
lyrics_embeddings = []
unsupported_tokens = set()
label_embedding_map = {} # dict{str: np.array([])}
count = 0
for i, lyric in tqdm(enumerate(lyrics)):
    labels = raw_df.loc[i]["labels"]
    labels = [label.strip() for label in labels.split(",")]
    print(labels)
    lyric_vector = np.zeros(300)
    for token in tokenize(lyric):
        try:
            lyric_vector += wv_from_bin.get_vector(token.lower())
        except KeyError as e:
            # if the word is not present in the glove then key error is raised, so handle the exception and move on
            unsupported_tokens.add(token)
            continue

    for label in labels:
        if label in label_embedding_map:
            label_embedding_map[label] += lyric_vector
        else:
            label_embedding_map[label] = lyric_vector

17it [00:00, 136.50it/s]

['Calmness', 'Sadness']
['Power', 'Tension']
['Amazement', 'Calmness', 'Solemnity', 'Tenderness']
['Joyful activation', 'Sadness', 'Tension']
['Amazement', 'Calmness', 'Joyful activation']
['Sadness', 'Solemnity', 'Tension']
['Amazement', 'Joyful activation', 'Solemnity', 'Tenderness']
['Nostalgia', 'Power', 'Sadness', 'Tenderness']
['Amazement', 'Joyful activation', 'Tenderness']
['Sadness', 'Tenderness', 'Tension']
['Calmness', 'Nostalgia', 'Power', 'Sadness', 'Solemnity']
['Nostalgia', 'Sadness', 'Tension']
['Nostalgia', 'Sadness', 'Tenderness']
['Joyful activation', 'Power', 'Solemnity', 'Tension']
['Power', 'Sadness', 'Solemnity', 'Tension']
['Joyful activation', 'Power', 'Solemnity']
['Sadness', 'Tenderness']
['Nostalgia', 'Sadness', 'Tenderness', 'Tension']
['Joyful activation', 'Power', 'Solemnity']
['Joyful activation', 'Nostalgia', 'Power']
['Calmness', 'Joyful activation', 'Nostalgia', 'Tenderness']
['Sadness', 'Tenderness', 'Tension']
['Sadness', 'Tenderness']
['Nostalgia',

66it [00:00, 201.78it/s]

['Calmness', 'Power', 'Sadness', 'Solemnity', 'Tension']
['Amazement', 'Joyful activation', 'Tenderness']
['Power', 'Solemnity']
['Power', 'Sadness', 'Solemnity']
['Nostalgia', 'Power', 'Tension']
['Amazement', 'Tenderness']
['Power', 'Tension']
['Amazement', 'Joyful activation', 'Nostalgia', 'Sadness', 'Solemnity', 'Tension']
['Sadness', 'Tenderness', 'Tension']
['Calmness', 'Joyful activation', 'Sadness', 'Solemnity']
['Power', 'Solemnity', 'Tension']
['Nostalgia', 'Sadness', 'Tension']
['Calmness', 'Power', 'Solemnity', 'Tension']
['Amazement', 'Tension']
['Amazement', 'Joyful activation', 'Nostalgia', 'Sadness', 'Tenderness']
['Nostalgia', 'Power', 'Sadness']
['Tenderness']
['Nostalgia', 'Sadness']
['Amazement', 'Joyful activation', 'Nostalgia', 'Power', 'Sadness']
['Joyful activation', 'Power', 'Solemnity']
['Nostalgia', 'Tenderness', 'Tension']
['Nostalgia', 'Sadness', 'Tenderness']
['Power', 'Solemnity', 'Tension']
['Calmness', 'Joyful activation', 'Tenderness']
['Sadness', 'Sol

122it [00:00, 247.21it/s]

...

['Sadness', 'Tension']
['Amazement', 'Joyful activation', 'Power', 'Tenderness', 'Tension']
['Amazement', 'Calmness', 'Solemnity', 'Tenderness']
['Calmness', 'Power', 'Sadness', 'Solemnity']
['Calmness', 'Nostalgia', 'Power', 'Tenderness']
['Joyful activation', 'Nostalgia']
['Calmness', 'Solemnity']
['Amazement', 'Joyful activation', 'Power']
['Nostalgia', 'Sadness', 'Tenderness']
['Sadness', 'Tenderness']
['Power', 'Sadness', 'Solemnity']
['Amazement', 'Calmness', 'Nostalgia', 'Power', 'Solemnity', 'Tension']
['Joyful activation', 'Power', 'Tension']
['Nostalgia', 'Sadness', 'Tenderness', 'Tension']
['Nostalgia', 'Sadness', 'Tension']
['Amazement', 'Joyful activation', 'Nostalgia', 'Sadness', 'Tension']
['Sadness']
['Solemnity', 'Tenderness']
['Nostalgia', 'Sadness', 'Tenderness']
['Amazement', 'Calmness', 'Joyful activation', 'Tenderness']
['Calmness', 'Nostalgia', 'Power', 'Sadness', 'Tenderness']
['Amazement', 'Calmness', 'Solemnity']
['Nostalgia', 'Tenderness', 'Tension']
['Calmne

1090it [00:04, 285.14it/s]

['Calmness', 'Joyful activation', 'Power', 'Solemnity']
['Calmness', 'Sadness', 'Tenderness', 'Tension']
['Joyful activation', 'Tenderness']
['Nostalgia', 'Sadness', 'Tenderness', 'Tension']
['Amazement', 'Calmness', 'Joyful activation', 'Solemnity']
['Sadness', 'Tenderness', 'Tension']
['Power', 'Tension']
['Calmness', 'Joyful activation', 'Nostalgia', 'Power', 'Sadness', 'Solemnity', 'Tension']
['Amazement', 'Calmness', 'Joyful activation', 'Solemnity']
['Nostalgia', 'Sadness', 'Solemnity', 'Tenderness']
['Nostalgia', 'Sadness', 'Solemnity', 'Tension']
['Joyful activation', 'Nostalgia', 'Power', 'Tension']
['Nostalgia', 'Solemnity', 'Tenderness']
['Nostalgia', 'Sadness', 'Solemnity', 'Tenderness']
['Sadness', 'Solemnity']
['Nostalgia', 'Sadness', 'Solemnity', 'Tenderness']
['Calmness', 'Nostalgia', 'Sadness']
['Amazement', 'Joyful activation', 'Power', 'Tenderness', 'Tension']
['Calmness', 'Nostalgia', 'Sadness', 'Tenderness']
['Joyful activation', 'Sadness', 'Solemnity', 'Tension']


1160it [00:04, 262.69it/s]

['Power', 'Sadness', 'Tension']
['Amazement', 'Calmness', 'Joyful activation', 'Power', 'Solemnity']
['Nostalgia', 'Sadness', 'Solemnity', 'Tension']
['Calmness', 'Nostalgia', 'Sadness', 'Solemnity', 'Tension']
['Power', 'Tension']
['Nostalgia', 'Power', 'Sadness', 'Solemnity']
['Joyful activation', 'Solemnity', 'Tension']
['Sadness', 'Solemnity', 'Tenderness']
['Joyful activation', 'Nostalgia', 'Power', 'Sadness', 'Tension']
['Nostalgia', 'Sadness', 'Solemnity', 'Tenderness']
['Sadness', 'Tenderness']
['Calmness', 'Joyful activation', 'Nostalgia']
['Nostalgia', 'Power', 'Sadness', 'Solemnity', 'Tension']
['Power', 'Tension']
['Power', 'Tension']
['Sadness']
['Power', 'Sadness', 'Tension']
['Amazement', 'Calmness', 'Nostalgia', 'Solemnity', 'Tenderness']
['Amazement', 'Joyful activation', 'Tenderness']
['Power', 'Tension']
['Nostalgia', 'Sadness', 'Tenderness']
['Nostalgia', 'Power', 'Solemnity']
['Power', 'Tension']
['Nostalgia', 'Sadness', 'Tenderness']
['Amazement', 'Joyful activati

In [274]:
count

0

In [275]:
emotions_embeddings = []
for k,v in label_embedding_map.items():
    v = v / np.linalg.norm(v, axis=0)
    label_embedding_map[k] = v
    emotions_embeddings.append(v)

emotions_embeddings = np.stack(emotions_embeddings)

In [276]:
emotions_embeddings.shape

(9, 300)

In [277]:
test_lyric = """
It might seem crazy what I am 'bout to say
Sunshine, she's here, you can take a break
I'm a hot air balloon that could go to space
With the air, like I don't care, baby by the way
Huh (Because I'm happy)
Clap along if you feel like a room without a roof
(Because I'm happy)
Clap along if you feel like happiness is the truth
(Because I'm happy)
Clap along if you know what happiness is to you
(Because I'm happy)
Clap along if you feel like that's what you wanna do
Here come bad news talking this and that (Yeah)
Well give me all you got, don't hold back (Yeah)
Well I should probably warn you I'll be just fine (Yeah)
No offense to you don't waste your time
Here's why
Clap along if you feel like a room without a roof
(Because I'm happy)
Clap along if you feel like happiness is the truth
(Because I'm happy)
Clap along if you know what happiness is to you
(Because I'm happy)
Clap along if you feel like that's what you wanna do
Uh, bring me down
Can't nothing, bring me down
My level's too high to bring me down
Can't nothing, bring me down, I said
Bring me down, can't nothing
Bring me down
My level's too high to bring me down
Can't nothing, bring me down, I said
Clap along if you feel like a room without a roof
(Because I'm happy)
Clap along if you feel like happiness is the truth
(Because I'm happy)
Clap along if you know what happiness is to you
(Because I'm happy)
Clap along if you feel like that's what you wanna do
Clap along if you feel like a room without a roof
(Because I'm happy)
Clap along if you feel like happiness is the truth
(Because I'm happy)
Clap along if you know what happiness is to you
(Because I'm happy)
Clap along if you feel like that's what you wanna do
Uh, bring me down (Happy, happy, happy, happy)
Can't nothing (Happy, happy, happy, happy)
Bring me down, my level's too high
To bring me down (Happy, happy, happy, happy)
Can't nothing (Happy, happy, happy, happy)
Bring me down, I said
Clap along if you feel like a room without a roof
(Because I'm happy)
Clap along if you feel like happiness is the truth
(Because I'm happy)
Clap along if you know what happiness is to you (ayy, ayy, ayy)
(Because I'm happy)
Clap along if you feel like that's what you wanna do
Clap along if you feel like a room without a roof
(Because I'm happy)
Clap along if you feel like happiness is the truth
(Because I'm happy)
Clap along if you know what happiness is to you (hey)
(Because I'm happy)
Clap along if you feel like that's what you wanna do
Come on
"""

In [278]:
test1_vector = vectorise(raw_df.loc[0]["lyrics"])
cs = distance(metric="cosine", embedding_matrix=emotions_embeddings, test_vector=test1_vector)

In [281]:
np.array(list(label_embedding_map.keys()))[(cs).argsort()[:3]]

array(['Joyful activation', 'Power', 'Tension'], dtype='<U17')

In [282]:
np.save("../embeddings/emotions_embeddings_v1.npy", emotions_embeddings)

In [283]:
label_embedding_map.keys()

dict_keys(['Calmness', 'Sadness', 'Power', 'Tension', 'Amazement', 'Solemnity', 'Tenderness', 'Joyful activation', 'Nostalgia'])